# Cosmos Db with container app
This Bicep template deploys an Azure Cosmos DB account with customizable settings, including location, database configurations, containers, failover options, and backup policies. It automates the setup of a Cosmos DB account, a SQL database, containers, and role assignments, ensuring flexibility and multi-region support.

## Prerequisites
- Azure CLI installed
- Bicep CLI installed
- Python
- Jupyter Notebook or JupyterLab installed


## Setup Environment
1. Make an .env file, configure the settings on your needs: 

In [21]:
%%writefile .env
SUBSCRIPTION_ID=<subscription_id>
RESOURCE_GROUP=<resource_group>
LOCATION=northeurope

Overwriting .env


2. Install `python-dotenv` if you haven't already. You can install it using pip:

In [ ]:
!pip install python-dotenv

3. Load the settings in the environments:

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
location = os.getenv('LOCATION')

deployment_name = "cosmos-db-deployment"
template_file = "main.bicep"
parameters_file = "main.bicepparam"

print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Location: {location}")

Subscription ID: 7ba9e6b6-7591-4828-aaa5-c4656cb871c3
Resource Group: cosmos-container-app
Location: eastus


4. Create an Azure Resource group:

In [2]:
!az login
!az account set --subscription  {subscription_id}
!az group create --name {resource_group} --location {location}

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "id": "835ebe2e-68d1-4817-925d-1aae42ec01db",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "37ac251a-9341-4ae8-b2a2-aa7febf5ce05"
      }
    ],
    "name": "RBR_DPT_GENERIC_P",
    "state": "Enabled",
    "tenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "user": {
      "name": "ROB5207@robeco.nl",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "id": "e84d14c9-e150-4500-93b2-5aa2829e295d",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "37ac251a-9341-4ae8-b2a2-aa7febf5ce05"
      }
    ],
    "name": "RBR_CSS_EQUITY_O",
    "state": "Enabled",
    "tenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "user": {
      "name": "ROB5207@robeco.nl",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "7

{
  "id": "/subscriptions/7ba9e6b6-7591-4828-aaa5-c4656cb871c3/resourceGroups/cosmos-container-app",
  "location": "eastus",
  "managedBy": null,
  "name": "cosmos-container-app",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


## Deploy template
1. Set the bicep parameters, adjust the settings when needed:

In [3]:
%%writefile {parameters_file}

using 'main.bicep'

var databaseName = 'testdb'

param cosmosDbConfig = {
  accountName: take('csmdb${uniqueString(databaseName)}', 15)
  database: {
    name: databaseName
  }
  containers: [
    {
      name: 'movies'
      partitionKey: {
        kind: 'Hash'
        paths: [
          '/id'
        ]
      }
    }
    {
      name: 'movies_leases'
      partitionKey: {
        kind: 'Hash'
        paths: [
          '/id'
        ]
      }
    }
    {
      name: 'movies_by_genre'
      partitionKey: {
        kind: 'Hash'
        paths: [
          '/genre'
        ]
      }
    }
  ]
}

param containerAppConfig = {
  environment: {
    name: 'my-container-app-env'
    resourceGroup: 'container-apps'
  }
  name: 'my-cosmos-container-app'
  ingress: {
    allowInsecure: false
    clientCertificateMode: 'ignore'
    external: true
    targetPort: 80
    transport: 'auto'
    traffic: [
      {
        latestRevision: true
        weight: 100
      }
    ]
  }
  containers: [
    {
      image: 'mcr.microsoft.com/azuredocs/containerapps-helloworld:latest'
      name: 'hello-world-container'
      resources: {
        cpu: '0.25'
        memory: '0.5Gi'
      }
      env: [
        {
          name: 'TEST_VAR1'
          value: 'hello world'
        }
      ]
    }
  ]
  scale: {
    minReplicas: 1
    maxReplicas: 2
    rules: [
      {
        name: 'http-requests'
        http: {
          metadata: {
            concurrentRequests: '10'
          }
        }
      }
    ]
  }
}

Overwriting main.bicepparam


2. Deploy the template:

In [5]:
!az deployment group create --resource-group {resource_group} --template-file {template_file}  --parameters {parameters_file} --name {deployment_name} 


ERROR: {"status":"Failed","error":{"code":"DeploymentFailed","target":"/subscriptions/7ba9e6b6-7591-4828-aaa5-c4656cb871c3/resourceGroups/cosmos-container-app/providers/Microsoft.Resources/deployments/cosmos-db-deployment","message":"At least one resource deployment operation failed. Please list deployment operations for details. Please see https://aka.ms/arm-deployment-operations for usage details.","details":[{"code":"ResourceDeploymentFailure","target":"/subscriptions/7ba9e6b6-7591-4828-aaa5-c4656cb871c3/resourceGroups/cosmos-container-app/providers/Microsoft.Resources/deployments/cosmos-deployment","message":"The resource write operation failed to complete successfully, because it reached terminal provisioning state 'Failed'.","details":[{"code":"DeploymentFailed","target":"/subscriptions/7ba9e6b6-7591-4828-aaa5-c4656cb871c3/resourceGroups/cosmos-container-app/providers/Microsoft.Resources/deployments/cosmos-deployment","message":"At least one resource deployment operation failed. 